### Importing libraries

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorboard

### Ingesting Data

In [39]:
dataset = tf.data.Dataset.list_files('datasets/train.csv', name='train')
dataset

<_ShuffleDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [ ]:
train =  tf.data.Dataset.from_tensor_slices(dataset)

train

### Data Exploration

In [ ]:
train.info()

In [ ]:
train.describe()

### Variable Distribution

In [ ]:
train.columns

In [ ]:
# grid = sns.FacetGrid(train)
# grid.map_dataframe(sns.histplot)
# plt.savefig('histograms.jpg')

In [405]:
import tensorflow as tf
import datetime, os

In [406]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

4422102/4422102 [==============================] - 8s 2us/step


In [407]:
def create_model():
  return tf.keras.

In [408]:
def train_model():

  model = create_model()
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(x=x_train, 
            y=y_train, 
            epochs=5, 
            validation_data=(x_test, y_test), 
            callbacks=[tensorboard_callback])

train_model()

Epoch 1/5
1875/1875 [==============================] - 35s 17ms/step - loss: 0.4975 - accuracy: 0.8226 - val_loss: 0.4266 - val_accuracy: 0.8441
Epoch 2/5
1875/1875 [==============================] - 30s 16ms/step - loss: 0.3805 - accuracy: 0.8612 - val_loss: 0.3757 - val_accuracy: 0.8606
Epoch 3/5
1875/1875 [==============================] - 29s 16ms/step - loss: 0.3487 - accuracy: 0.8719 - val_loss: 0.3597 - val_accuracy: 0.8681
Epoch 4/5
1875/1875 [==============================] - 30s 16ms/step - loss: 0.3303 - accuracy: 0.8782 - val_loss: 0.3425 - val_accuracy: 0.8764
Epoch 5/5
1875/1875 [==============================] - 28s 15ms/step - loss: 0.3145 - accuracy: 0.8834 - val_loss: 0.4075 - val_accuracy: 0.8540
